# GP-LCCM Example
The purpose of this notebook is to demonstrate how the Gaussian Process - Latent Class Choice Model (GP-LCCM) code from Sfeir et al. (2021) could be used.

The application is based on the "London Dataset" from Hillel et al.(2018) which is available online as a supplementary material to the paper.

The dataset contains individual trip diaries of the London Travel Demand Survey (LTDS) from April 2012 to March 2015 with their corresponding modes alternatives extracted from a Google directions application programming interface (API) and corresponding estimates of car operating costs and public transport fares. The dataset consists of 81,086 trips, four modes (walke, cycle, public transport, and drive), and different trip purposes (e.g. Home-Based Work, Home-Based Education, etc.).

The required steps to build a travel mode choie model and run the GP-LCCM code successfully are listed below.

##### References
Sfeir, G., Rodrigues, F., Abou-Zeid, M. (2022). “Gaussian Process Latent Class Choice Models.” Transportation Research Part C: Emerging Technologies, 136, 103552. https://doi.org/10.1016/j.trc.2022.103552

Hillel, T., Elshafie, M.Z.E.B., Jin, Y. (2018). "Recreating passenger mode choice-sets for transport simulation: A case study of London, UK," in: Proceedings of the Institution of Civil Engineers - Smart Infrastructure and Construction

### Step 1: Import the GP-LCCM Package and All other Necessary Packages

In [1]:
import GP_LCCM
import numpy as np
import pandas as pd
import pylogit as pl
import warnings
from collections import OrderedDict
from sklearn import preprocessing

### Step 2: Load the Datasets
The GP-LCCM consists of two sub-models, a class memberhsip model and a class-specific choice model:

- The class membership model is defined as a Gaussian Process to cluster individuals probabilistically to K clusters (latent classes)

- Conditioned on the class membership of an individual, the class-specific choice model estimates the probability of choosing a specific alternative

The class-specific choice model needs the data to be in "long" format while the class membership model requires a "wide" format.

For more details regarding how to convert your data from wide to long format, you can refer to the following link: https://github.com/timothyb0912/pylogit/blob/master/examples/notebooks/Main%20PyLogit%20Example.ipynb

In [2]:
# load the "london" dataset, which is available in wide format
wide_london = pd.read_csv('dataset.csv')

# the .copy() returns a copy of the dataset and ensures that modifications are made to a copy and not the original data
wide_london = wide_london.copy()

In [3]:
wide_london.head()

,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,dur_pt_int_total,dur_pt_int_waiting,dur_pt_int_walking,pt_n_interchanges,dur_driving,cost_transit,cost_driving_total,cost_driving_fuel,cost_driving_con_charge,driving_traffic_percent
0,0,0,0,0,drive,HBO,Petrol_Car,full,1.0,1,...,0.0,0.0,0.0,0,0.052222,1.5,0.14,0.14,0.0,0.111702
1,1,0,0,1,drive,HBO,Petrol_Car,full,1.0,1,...,0.0,0.0,0.0,0,0.059444,1.5,0.15,0.15,0.0,0.112150
2,2,0,0,2,drive,HBO,Petrol_Car,full,1.0,1,...,0.0,0.0,0.0,0,0.236667,1.5,0.79,0.79,0.0,0.203052
3,3,0,0,3,drive,HBO,Petrol_Car,full,1.0,1,...,0.0,0.0,0.0,0,0.233333,1.5,0.78,0.78,0.0,0.160714
4,4,0,1,2,drive,HBO,Petrol_Car,dis,1.0,1,...,0.0,0.0,0.0,0,0.229167,1.5,0.78,0.78,0.0,0.130909


In [4]:
# 6 columns need to be added to the original dataset:
# person_id: a unique id for each person regardless of the household id (starts from 1)
# travel_mode_nb: 1 for walk, 2 for cycle, 3 for pt, and 4 for drive
# walk_av, cycle_av, pt_av, drive_av: to determine the availability of each mode (1 if available and 0 otherwise).
# For simplicity, we assume that all modes are available for all individuals

# Create unique person_id
wide_london['person_id'] = wide_london.groupby(['household_id', 'person_n']).ngroup() + 1

# travel_mode_nb: 1 for walk, 2 for cycle, 3 for pt, and 4 for drive
# walk_av, cycle_av, pt_av, drive_av: to determine the availability of each mode (1 if available and 0 otherwise).
# For simplicity, we assume that all modes are available for all individuals

# Define mapping
mode_mapping = {'walk': 1, 'cycle': 2, 'pt': 3, 'drive': 4}

# Apply mapping
wide_london['travel_mode_nb'] = wide_london['travel_mode'].map(mode_mapping)

wide_london['walk_av'] = 1
wide_london['cycle_av'] = 1
wide_london['pt_av'] = 1
wide_london['drive_av'] = 1

In [5]:
# for this application we will use a sample of 1000 individuals
wide_london = wide_london[wide_london['person_id'] < 1000]

# reset index 
wide_london.reset_index(inplace=True, drop=True)

In [6]:
# look at the first 5 rows of the data
wide_london.head(5).T

,0,1,2,3,4
trip_id,0,1,2,3,4
household_id,0,0,0,0,0
person_n,0,0,0,0,1
trip_n,0,1,2,3,2
travel_mode,drive,drive,drive,drive,drive
purpose,HBO,HBO,HBO,HBO,HBO
fueltype,Petrol_Car,Petrol_Car,Petrol_Car,Petrol_Car,Petrol_Car
faretype,full,full,full,full,dis
bus_scale,1.0,1.0,1.0,1.0,1.0
survey_year,1,1,1,1,1


### Step 3: Convert the data from wide to long format
We will rely in this section on the pylogit package which was developed to estimate many discrete choice models such as multinomial logit models, mixed logit and nested logit https://pypi.org/project/pylogit/. The package can be also used to convert the data from wide to long format.

In [7]:
# look at the columns of the london dataset
wide_london.columns

Index(['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_mode',
       'purpose', 'fueltype', 'faretype', 'bus_scale', 'survey_year',
       'travel_year', 'travel_month', 'travel_date', 'day_of_week',
       'start_time_linear', 'age', 'female', 'driving_license',
       'car_ownership', 'distance', 'dur_walking', 'dur_cycling',
       'dur_pt_total', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus',
       'dur_pt_int_total', 'dur_pt_int_waiting', 'dur_pt_int_walking',
       'pt_n_interchanges', 'dur_driving', 'cost_transit',
       'cost_driving_total', 'cost_driving_fuel', 'cost_driving_con_charge',
       'driving_traffic_percent', 'person_id', 'travel_mode_nb', 'walk_av',
       'cycle_av', 'pt_av', 'drive_av'],
      dtype='object')

In [8]:
# Create the list of individual specific variables (variables that are specific to an individual 
# and do not vary across alternatives)
ind_variables = ['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_mode',
       'purpose', 'fueltype', 'faretype', 'bus_scale', 'survey_year',
       'travel_year', 'travel_month', 'travel_date', 'day_of_week',
       'start_time_linear', 'age', 'female', 'driving_license',
       'car_ownership', 'person_id']


# Specify the variables that vary across individuals and some or all alternatives
# The keys are the column names that will be used in the long format dataframe
# The values are dictionaries whose key-value pairs are the alternative id and
# the column name of the corresponding column that encodes that variable for
# the given alternative. Examples below.
alt_varying_variables = {u'travel_time': dict([(1, 'dur_walking'),
                                               (2, 'dur_cycling'),
                                               (3, 'dur_pt_total'),
                                               (4, 'dur_driving')]),
                        u'travel_cost': dict([(3, 'cost_transit'),
                                              (4, 'cost_driving_total')])}

# Specify the availability variables
# Note that the keys of the dictionary are the alternative id's.
# The values are the columns denoting the availability for the
# given mode in the dataset.
availability_variables = {1: 'walk_av',
                          2: 'cycle_av',
                          3: 'pt_av',
                          4: 'drive_av'}


# Determine the columns for: alternative ids, the observation ids and the choice
# The 'custom_alt_id' is the name of a column to be created in the long-format data
# It will identify the alternative associated with each row.
custom_alt_id = "mode_id"

# Create a custom id column that ignores the fact that this is a 
# panel/repeated-observations dataset. Note the +1 ensures the id's start at one.
obs_id_column = "custom_id"
wide_london[obs_id_column] = np.arange(wide_london.shape[0],
                                            dtype=int) + 1


# Create a variable recording the choice column
choice_column = "travel_mode_nb"

In [9]:
# Perform the conversion to long-format
long_london = pl.convert_wide_to_long(wide_london, 
                                           ind_variables, 
                                           alt_varying_variables, 
                                           availability_variables, 
                                           obs_id_column, 
                                           choice_column,
                                           new_alt_id_name=custom_alt_id)
# Look at the resulting long-format dataframe
long_london.head(10).T

/Users/GEOSAF/Desktop/BiogemeEnv/lib/python3.12/site-packages/pylogit/choice_tools.py:454: UserWarning: Note, there are 43 variables in wide_data but the inputs ind_vars, alt_specific_vars, and subset_specific_vars only account for 32 variables.
  warnings.warn(msg.format(num_dataframe_vars, data_title) +


,0,1,2,3,4,5,6,7,8,9
custom_id,1,1,1,1,2,2,2,2,3,3
mode_id,1,2,3,4,1,2,3,4,1,2
travel_mode_nb,0,0,0,1,0,0,0,1,0,0
trip_id,0,0,0,0,1,1,1,1,2,2
household_id,0,0,0,0,0,0,0,0,0,0
person_n,0,0,0,0,0,0,0,0,0,0
trip_n,0,0,0,0,1,1,1,1,2,2
travel_mode,drive,drive,drive,drive,drive,drive,drive,drive,drive,drive
purpose,HBO,HBO,HBO,HBO,HBO,HBO,HBO,HBO,HBO,HBO
fueltype,Petrol_Car,Petrol_Car,Petrol_Car,Petrol_Car,Petrol_Car,Petrol_Car,Petrol_Car,Petrol_Car,Petrol_Car,Petrol_Car


### Step 4: Model Specification

#### Step 4.1: Class Membership

In [10]:
# set the number of clusters (latent classes) to 2
n_classes = 2

In [11]:
# create dummy variables for car ownership
wide_london['car_ownership0'] = (wide_london['car_ownership'] == 0).astype(int)
wide_london['car_ownership1'] = (wide_london['car_ownership'] == 1).astype(int)
wide_london['car_ownership2'] = (wide_london['car_ownership'] == 2).astype(int)

In [12]:
# the class membership model is defined as a Gaussian Process

# select the variables for the class membership model
X_Mem2 = wide_london[['person_id','age','female','driving_license','car_ownership1','car_ownership2']]

# remove duplicates
X_Mem2.drop_duplicates(subset ="person_id",keep= 'first', inplace = True)

X_Mem = X_Mem2[['age','female','driving_license','car_ownership1','car_ownership2']]

# it is a good practice to standardize the variables for the Gaussian Process Model
Standardized_X_Mem = preprocessing.scale(X_Mem)

/var/folders/m2/kskgzsyn6t9b8xsjgl711fv00000gp/T/ipykernel_7174/1145831177.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_Mem2.drop_duplicates(subset ="person_id",keep= 'first', inplace = True)


Different kernel or combination of kernel functions can be used for the Gaussian Process model. In this exmaple we use the following kernel function: DotProduct(sigma_0=1)

The kernel function can be changed from the GP_LCCM.py file under: #### kernel function.

For more details about the different kernel functions that can be used, you can refer to Section 1.7.5 on the following link:
https://scikit-learn.org/stable/modules/gaussian_process.html#kernels-for-gaussian-processes

#### Step 4.2: Class-Specific Choice Model
This section is based on the latent class choice model (lccm) package. For more details on how to specify the class-specific choice model, you can check the lccm package on the following link: https://github.com/ferasz/LCCM

You can use alternative specific or generic speficifation for the parameters. 
We will use in this example both types to demonstrate how each one can be used:

- intercept: alternative specific constants included in all the utilities except the one related to walk, hence the one braket with the numbers of the corresponding alternatives [2,3,4] (walk is used as a base)
- travel time parameters for both classes are specified as alternative specific and included in the utilities of all modes, hence the one braket with the numbers of the corresponding alternatives [1,2,3,4]
- travel cost parameters for both classes are specified as generic and included only in the utilities of drive and pt, hence the two brakets and the numbers of the alternatives [[3,4]]


The specification of the two classes is as follows:
##### Latent Class 1:

$V_{walk} =        \beta_{tt, class1} * TravelTime_{walk}  $  

$V_{cycle} = ASC-cycle_{class1} + \beta_{tt, class1} * TravelTime_{cycle}$

$V_{pt} = ASC-pt_{class1} + \beta_{tt, class1} * TravelTime_{pt} + \beta_{cost, class1} * TravelCost_{pt} $

$V_{drive} = ASC-drive_{class1} + \beta_{tt, class1} * TravelTime_{drive} + \beta_{cost, class1} * TravelCost_{drive} $

##### Latent Class 2:

$V_{walk} =        \beta_{tt, class2} * TravelTime_{walk}  $  

$V_{cycle} = ASC-cycle_{class2} + \beta_{tt, class2} * TravelTime_{cycle}$

$V_{pt} = ASC-pt_{class2} + \beta_{tt, class2} * TravelTime_{pt} + \beta_{cost, class2} * TravelCost_{pt} $

$V_{drive} = ASC-drive_{class2} + \beta_{tt, class2} * TravelTime_{drive} + \beta_{cost, class2} * TravelCost_{drive} $

In [13]:
# scale travel time from hours to minutes
long_london['travel_time'] = long_london['travel_time'] * 60

# add a column of ones named 'intercept'. This will be used later to define the Alternative-Specific Constants (ASCs).
long_london['intercept']=np.ones(long_london.shape[0])

In [14]:
long_london.columns

Index(['custom_id', 'mode_id', 'travel_mode_nb', 'trip_id', 'household_id',
       'person_n', 'trip_n', 'travel_mode', 'purpose', 'fueltype', 'faretype',
       'bus_scale', 'survey_year', 'travel_year', 'travel_month',
       'travel_date', 'day_of_week', 'start_time_linear', 'age', 'female',
       'driving_license', 'car_ownership', 'person_id', 'travel_time',
       'travel_cost', 'intercept'],
      dtype='object')

In [15]:
# NOTE: Specification and variable names must be in lists of ordered dictionaries.
# class_specific_specs defines the variables names to be used in the specification of the 
# class specific choice model of each class.
# class_specific_labels defines the names associated with each of the variables which
# will be displayed in the output tables after estimation.


class_specific_specs = [OrderedDict([('intercept',[2,3,4]),
                                     ('travel_time',[1,2,3,4]),
                                     ('travel_cost',[[3,4]])]),
                        OrderedDict([('intercept',[2,3,4]),
                                     ('travel_time',[1,2,3,4]),
                                     ('travel_cost',[[3,4]])])]
                       
class_specific_labels = [OrderedDict([('ASC',['ASC(cycle)',
                                             'ASC(pt)',
                                             'ASC(drive)',]),
                                      ('travel_time',['Travel Time(walk)',
                                                      'Travel Time(cycle)',
                                                      'Travel Time(pt)',
                                                      'Travel Time(drive)']),
                                      ('travel_cost',['Travel Cost'])]),
                         OrderedDict([('ASC',['ASC(cycle)',
                                             'ASC(pt)',
                                             'ASC(drive)',]),
                                      ('travel_time',['Travel Time(walk)',
                                                      'Travel Time(cycle)',
                                                      'Travel Time(pt)',
                                                      'Travel Time(drive)']),
                                      ('travel_cost',['Travel Cost'])])]

In [16]:
# starting values for the Class-Specific Choice Model paramerters
paramClassSpec = []
paramClassSpec.append(np.array([0,0,0,0,0,0,0,0])) # for the first class
paramClassSpec.append(np.array([0,0,0,0,0,0,0,0])) # for the second class

### Step 5: Model Estimation and Results

To train/estimate the model, the following must be included in the GP_LCCM.lccm_fit function:
- data: data used for the class-speficic choice model in long format
- X: variables used for the class membership model in wide format

For prediction/test, the following must be included in the GP_LCCM.lccm_fit function:
- dataTest = data for the class-speficic choice model in long format
- XTest = variables used for the class membership model in wide format

Note that after training your model you can test it on a different dataset (test dataset). For this example, we use the same one that was used in training. However, you can split the data into train and test datasets(e.g. 80%-20%), estimate the model on the train dataset (data, X) and test the model on the test dataset (dataTest, XTest). The model will generate the prediction accuracy in terms of LL at the end of the notebook (Predicted Log-Likelihood).

The remaining variables that must be included in the GP_LCCM.lccm_fit function are:
- prediction_test: 'Yes' if you would like to test the model on a test dataset and 'No' otherwise
- ind_id_col: name of column identifying the individual for each row of data 
- obs_id_col: name of column identifying the observation (choice scenario)
- choice_col: name of column identifying whether the alternative corresponding by a row was chosen or not
- n_classes: number of latent classes (clusters)
- paramClassSpec: starting values for the class-specific choice models. If paramClassSpec is None, the model will generate the starting values randomly

In [17]:
# define a location to store the resutls 
outputFilePath = '.../Results/'

# run the model 
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    
    GP_LCCM.lccm_fit(data = long_london,
                             X = Standardized_X_Mem,
                             dataTest = long_london,
                             XTest = Standardized_X_Mem,
                             prediction_test = 'Yes',
                             ind_id_col = 'person_id', 
                             obs_id_col = 'custom_id',
                             alt_id_col = 'mode_id',
                             choice_col = 'travel_mode_nb', 
                             n_classes = n_classes,
                             reg_covar = 1e-6,
                             tol = 1e-3,
                             max_iter = 100,
                             class_specific_specs = class_specific_specs,
                             class_specific_labels = class_specific_labels,
                             #paramClassSpec = paramClassSpec,
                             outputFilePath = outputFilePath,
                             outputFileName = 'ModelResults')


Processing data
Initializing EM Algorithm...

<Tue, 11 Feb 2025 14:35:41> Iteration 0: -3119.0130
<Tue, 11 Feb 2025 14:35:42> Iteration 1: -2156.8601
<Tue, 11 Feb 2025 14:35:43> Iteration 2: -2080.6024
<Tue, 11 Feb 2025 14:35:44> Iteration 3: -2008.1784
<Tue, 11 Feb 2025 14:35:45> Iteration 4: -1914.8490
<Tue, 11 Feb 2025 14:35:45> Iteration 5: -1816.6073
<Tue, 11 Feb 2025 14:35:45> Iteration 6: -1733.7329
<Tue, 11 Feb 2025 14:35:46> Iteration 7: -1667.7187
<Tue, 11 Feb 2025 14:35:48> Iteration 8: -1631.7268
<Tue, 11 Feb 2025 14:35:48> Iteration 9: -1618.8634
<Tue, 11 Feb 2025 14:35:48> Iteration 10: -1616.2394
<Tue, 11 Feb 2025 14:35:50> Iteration 11: -1613.8252
<Tue, 11 Feb 2025 14:35:50> Iteration 12: -1611.8262
<Tue, 11 Feb 2025 14:35:51> Iteration 13: -1610.3688
<Tue, 11 Feb 2025 14:35:52> Iteration 14: -1609.3526
<Tue, 11 Feb 2025 14:35:53> Iteration 15: -1608.6399
<Tue, 11 Feb 2025 14:35:53> Iteration 16: -1608.0656
<Tue, 11 Feb 2025 14:35:54> Iteration 17: -1607.8427
<Tue, 11 F

In [18]:
# initial kernel function
GP_LCCM.gpc_model.kernel

DotProduct(sigma_0=1)

In [19]:
# estimated kernel function
GP_LCCM.gpc_model.kernel_

DotProduct(sigma_0=0.406)